In [1]:
from docuscospacy import convert_corpus,frequency_table, tags_table, ngrams_by_token, ngrams_by_tag, coll_table, \
    tags_dtm, normalize_dtm, dtm_to_coo, kwic_center_node, keyness_table, tag_ruler
from tmtoolkit.corpus import Corpus, vocabulary_size, corpus_num_tokens, corpus_add_tabular

import os
import re
import spacy
import pandas as pd 
import random


OSError: [E050] Can't find model 'en_docusco_spacy'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def csv_2_txt(inpath, outdir): #where inpath is local path to data csv, outdir to folder for split txt files

    df = pd.read_csv(inpath, sep = "\t", engine = "python", on_bad_lines= "warn")

    df = df.drop(["User", "Date Created", "Source of Tweet", "Location", "Number of Likes"], axis = 1)

    df["Tweet"] = df["Tweet"].str.lower()
    
    #n-grams advertising in the corpus. used to filter out advertisments. 
    ad_grams = ["super affordable", "place your order", "giveaway", "asamoah gyan", "black stars", "baby jet", "ghanavsnigeria", "forextradingbot", "intelligence prime capital", "#gistlover", "kindly send a dm", "countdown", "stream", "#breakingbad", "#freesubscription", "dstv", "#dstv", "don jazzy", "good condition", "agba akin", "referral program", "#cryptocurrency", "multichoice", "#nugaunilag2022", "snapchat", "spintex", "#blackstars", "#skincareproducts", "#worldwaterday", "today's bookings", "#endsars", "oppong nkrumah", "for enquiries", "#tech tuesdays", "promote my business", "just visit the website", "bbnaija", "bbn", "speed darlington", "#sheleadsafrica", "daniel emeka", "trending health", "wellness gadget", "apple music", "follow back", "for sale", "#forsale"]

    df_no_ads = df[~df['Tweet'].str.contains('|'.join(ad_grams))]
    
    for i, value in enumerate(df_no_ads["Tweet"]):
        filename = os.path.join(f'{outdir}/{random.randint(10000, 99999)}.txt')
        with open(filename, "w", encoding="utf-8") as f:
            f.write(str(value))



In [ ]:
nlp = spacy.load("en_docusco_spacy")

def pre_process(txt):
    txt = re.sub(r'\bits\b', 'it s', txt)
    txt = re.sub(r'\bIts\b', 'It s', txt)
    txt = " ".join(txt.split())
    return(txt)


def build_corpus(filepath): #where filpath is outpath from previous cell
    
    corpus = Corpus.from_folder(filepath,
                              spacy_instance = nlp,
                              raw_preproc = [pre_process],
                              spacy_token_attrs = ['tag', 'ent_iob', 'ent_type', 'is_punct'])

    return corpus     

In [2]:
#this cell processes the corpus

target_corpus = build_corpus("data/target_corpus.txt") #replace filepath with your local path
tar_corpus_total = corpus_num_tokens(target_corpus)
tar_corpus_types = vocabulary_size(target_corpus)
tar_total_punc = []

for i in range(0,len(target_corpus)):
    tar_total_punc.append(sum(target_corpus[i]['is_punct']))
    
tar_total_punct = sum(tar_total_punc)
tar_non_punct = tar_corpus_total - tar_total_punct

processed_target = convert_corpus(target_corpus)

In [5]:
#this cell contains functions for different analyses, like frequency, specific type counts, and ngrams. all functions can run with var_type 'pos' or 'ds'

def get_high_freq(corpus, var_type):  
    wc = frequency_table(corpus, count_by=var_type)
    high_rf = wc[wc['RF'] > 100]
    
    return high_rf

def get_tag_list(corpus, var_type):
    dc = tags_table(corpus, count_by=var_type)
    dc = dc.sort_values('RF', ascending = False)
    
    return dc

def get_type_freq(corpus, type_name, var_type): #specify a corpus and a docuscope category and this function will return all tokens in that type
    docu_wc = frequency_table(corpus, count_by=var_type)
    type_freq_wc = docu_wc.query(f'Tag.str.starswith{type_name}')
    
    return type_freq_wc

def get_ngrams_by_token(corpus, token, node_pos, span, var_type): #node pos = where in ngram is the token
    
    nc = ngrams_by_token(corpus, node_word=token, node_position=node_pos, span=span, search_type='fixed', count_by=var_type)

    return nc


done
